In [ ]:
import pandas as pd
import numpy as np

from scipy.ndimage import gaussian_filter
from skimage import metrics

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [ ]:
cover_img = plt.imread('/kaggle/input/miniatures/M3-1-O.jpg')

print("Cover Reference Image")
plt.figure(figsize=(8,8))
plt.imshow(cover_img)
plt.axis('off');

# grid = (int(100 * cover_img.shape[0] / cover_img.shape[1]), 100)
# print(grid)

In [ ]:
def calculate_ssim(par1, par2):
    ssim = metrics.structural_similarity(par1, par2, multichannel=False, data_range = max(par1.max(), par2.max()) - min(par1.min(), par2.min()))
    return ssim

def calculate_mse(par1, par2):
    mse = np.mean((par1 - par2) ** 2)
    return mse

In [ ]:
def export_heatmap(par = pd.DataFrame()):
    
    par = par[(par.on_surf == True) & (par.confidence > 0.8)]
#     grid = np.divide(cover_img.shape[0:2], 150).astype(int)
    grid = (int(10 * cover_img.shape[0] / cover_img.shape[1]), 10)
    heatmap_detail = 0.05 # this will determine the gaussian blur kernel of the image (higher number = more blur)
    
    # flip the fixation points
    # from the original coordinate system,
    # where the origin is at botton left,
    # to the image coordinate system,
    # where the origin is at top left
    gaze_on_surf_x = par['x_norm']
    gaze_on_surf_y = par['y_norm']
    gaze_on_surf_y = 1 - gaze_on_surf_y
    
    # make the histogram
    hist, x_edges, y_edges = np.histogram2d(
        gaze_on_surf_y,
        gaze_on_surf_x,
        range=[[0, 1.0], [0, 1.0]],
        density=False,
        bins=grid
    )
    
    # gaussian blur kernel as a function of grid/surface size
    filter_h = int(heatmap_detail * grid[0]) // 2 * 2 + 1
    filter_w = int(heatmap_detail * grid[1]) // 2 * 2 + 1
    heatmap = gaussian_filter(hist, sigma=(filter_w, filter_h), order=0)

    return heatmap

In [ ]:
def group_similarity_df_builder(painting_groups, painting_number = 1):
    
    painting_group_df = {}

    painting_group_df['group_name'] = []
    painting_group_df['ssi'] = []
    painting_group_df['mse'] = []
    painting_group_df['kind'] = []

    for gp in painting_groups:
        
        painting_group_df['group_name'].append(gp[0] + "_" + gp[1])
        par1_pair = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_pair_'+ str(painting_number) +'/' + gp[0] +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')
        par2_pair = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_pair_'+ str(painting_number) +'/' + gp[1] +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')

        painting_group_df['group_name'].append(gp[0] + "_" + gp[1])
        par1_ind = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_individual_'+ str(painting_number) +'/' + gp[0] +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')
        par2_ind = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_individual_'+ str(painting_number) +'/' + gp[1] +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')

        heatmap1_pair, heatmap2_pair = export_heatmap(par = par1_pair), export_heatmap(par = par2_pair)

        ssi_pair_index = calculate_ssim(heatmap1_pair, heatmap2_pair)
        painting_group_df['ssi'].append(ssi_pair_index)

        mse_pair_index = calculate_mse(heatmap1_pair, heatmap2_pair)
        painting_group_df['mse'].append(mse_pair_index)

        painting_group_df['kind'].append('Pair')

        heatmap1_ind, heatmap2_ind = export_heatmap(par = par1_ind), export_heatmap(par = par2_ind)

        ssi_ind_index = calculate_ssim(heatmap1_ind, heatmap2_ind)
        painting_group_df['ssi'].append(ssi_ind_index)

        mse_pair_index = calculate_mse(heatmap1_ind, heatmap2_ind)
        painting_group_df['mse'].append(mse_pair_index)

        painting_group_df['kind'].append('Individual')
        
    return pd.DataFrame(painting_group_df)

In [ ]:
painting1_groups = [['par1','par2'],['par5','par6'],['par7','par8'],['par9','par10'],['par11','par12'],['par13','par14']]
painting_number = '4'

painting1_group_df = group_similarity_df_builder(painting_groups = painting1_groups, painting_number=painting_number,)

In [ ]:
sns.pointplot(data=painting1_group_df, y="ssi", x="kind", order = ['Individual', 'Pair'], hue="group_name")
plt.ylabel('Common Ground Similarity')
plt.xlabel('Session')
plt.title('Painting #4 - SSI')

plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
stats.ttest_rel(painting1_group_df[painting1_group_df.kind == 'Individual'].ssi, painting1_group_df[painting1_group_df.kind == 'Pair'].ssi)

In [ ]:
sns.pointplot(data=painting1_group_df, y="ssi", x="kind", order = ['Individual', 'Pair'], hue="group_name")
plt.ylabel('Common Ground Similarity')
plt.xlabel('Session')
plt.title('Painting #4 - SSI')

plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
stats.ttest_rel(painting1_group_df[painting1_group_df.kind == 'Individual'].ssi, painting1_group_df[painting1_group_df.kind == 'Pair'].ssi)

In [ ]:
# sns.pointplot(data=painting1_group_df, y="mse", x="kind", order = ['Individual', 'Pair'], hue="group_name")
# plt.ylabel('Common Ground Similarity')
# plt.xlabel('Session')
# plt.title('Painting #1 - MSE')

# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# stats.ttest_rel(painting1_group_df[painting1_group_df.kind == 'Individual'].mse, painting1_group_df[painting1_group_df.kind == 'Pair'].mse)

In [ ]:
# painting2_groups = [ ['par5','par6'],['par7','par8'],['par9','par10'],['par11','par12'],['par13','par14']]
# painting_number = '2'

# painting2_group_df = group_similarity_df_builder(painting_groups = painting2_groups,painting_number=painting_number)

In [ ]:
# painting2_group_df = group_similarity_df_builder(painting_groups = painting2_groups,painting_number=painting_number)
# sns.pointplot(data=painting2_group_df, y="ssi", x="kind", order = ['Individual', 'Pair'], hue="group_name")
# plt.ylabel('Common Ground Similarity')
# plt.xlabel('Session')
# plt.title('Painting #2-SSI')

# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# stats.ttest_rel(painting2_group_df[painting2_group_df.kind == 'Individual'].mse, painting2_group_df[painting2_group_df.kind == 'Pair'].mse)

In [ ]:
# painting2_group_df = group_similarity_df_builder(painting_groups = painting2_groups,painting_number=painting_number)
# sns.pointplot(data=painting2_group_df, y="mse", x="kind", order = ['Individual', 'Pair'], hue="group_name")
# plt.ylabel('Common Ground Similarity')
# plt.xlabel('Session')
# plt.title('Painting #2-MSE')

# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# stats.ttest_rel(painting2_group_df[painting2_group_df.kind == 'Individual'].mse, painting2_group_df[painting2_group_df.kind == 'Pair'].mse)

In [ ]:
painting_number = 9

gp1 = 'par11'
gp2 = 'par12'

par1_pair = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_pair_'+ str(painting_number) +'/' + gp1 +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')
par2_pair = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_pair_'+ str(painting_number) +'/' + gp2 +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')
par1_ind = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_individual_'+ str(painting_number) +'/' + gp1 +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')
par2_ind = pd.read_csv('/kaggle/input/modified-csv-individual-and-pair/csv/csv/stim0' + str(painting_number) +'/modified_individual_'+ str(painting_number) +'/' + gp2 +'_modified_gaze_positions_on_surface_'+ str(painting_number) +'.csv')

heatmap1_pair, heatmap2_pair = export_heatmap(par = par1_pair), export_heatmap(par = par2_pair)
heatmap1_ind, heatmap2_ind = export_heatmap(par = par1_ind), export_heatmap(par = par2_ind)

In [ ]:
print(heatmap1_pair)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

axes[0, 0].imshow(heatmap1_ind, cmap='jet', interpolation='nearest')
axes[0, 0].set_title('par#11 Individual')

axes[0, 1].imshow(heatmap2_ind, cmap='jet', interpolation='nearest')
axes[0, 1].set_title('par#12 Individual')

axes[1, 0].imshow(heatmap1_pair, cmap='jet', interpolation='nearest')
axes[1, 0].set_title('par#11 Pair')

axes[1, 1].imshow(heatmap2_pair, cmap='jet', interpolation='nearest')
axes[1, 1].set_title('par#12 Pair')

plt.tight_layout()
plt.show()


